# =================初始化==============

In [3]:
import networkx as nx
import re
import copy
import time

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET


G = nx.DiGraph()
id = 0
title_page = 0
#init

In [4]:
st = time.time()

for round in range(1,11):
    dic={}
    
    print("===============reading document enfull"+str(round)+"================")
    dom = ET.parse("enfull"+str(round)+".xml")

    root = dom.getroot()
    print(root)
    if root.tag.find("}"):
        start = root.tag.find("{")
        end = root.tag.find("}")
        front = root.tag[start:end+1]
    else:
        front = ''
    
    for i in root.iter(front+'page'):
        #print(i.find('title').text)
        pattern = re.compile(r'\[\[[\s\w\|\(\)&;]*\]\]')
        dd = pattern.findall(str(i.find(front+'revision').find(front+'text').text))
        dic[i.find(front+'title').text] = dd
        
    for i in dic:
        lst = []
        for j in dic[i]:
            j = j.replace("[",'')
            j = j.replace("]",'')
            lst.append(j)
        #print(lst)
        dic[i] = lst 
#================================================================
    for i in dic:
        count = 0
        for j in dic[i]:
            if j.find(" ") != -1:
                dic[i][count] = dic[i][count].replace(" ","_")
            if j.find("|") != -1:
                end = j.find("|")
                dic[i][count] = dic[i][count][:end]
            count = count + 1
            
    lst = dic.keys()
    lst = list(lst)
    for i in range(len(dic)):
        if lst[i].find(" ") != -1:
            j = lst[i].replace(" ","_")
            dic[j] = dic.pop(lst[i])
#================================================================

    print("title_page change")
    print(title_page)
    title_page = title_page + len(dic.keys())
    print(title_page)
    print("==================================")
    
    print("id change")
    print(id)
    for i in dic:
        if i not in G.nodes:
            G.add_node(i)
            G.nodes[i]["id"] = id
            id = id + 1
        for j in dic.get(i):
            if j not in G.nodes:
                G.add_node(j)
                G.add_edge(i,j)
                G.nodes[j]["id"] = id 
                id = id + 1

    print(id)
    print("==================================")

et = time.time()

print("using %f time to reading" %(et-st))
#print(G.nodes.data())

# ==================================

===============reading document enfull1================
<Element 'mediawiki' at 0x00000125884FC720>
title_page change
0
27400
id change
0
1043398
===============reading document enfull2================
<Element 'mediawiki' at 0x0000012608F03B30>
title_page change
27400
110992
id change
1043398
1888173
===============reading document enfull3================
<Element 'mediawiki' at 0x000001272D908EA0>
title_page change
110992
200893
id change
1888173
2692926
===============reading document enfull4================
<Element 'mediawiki' at 0x00000127A4BAD680>
title_page change
200893
341641
id change
2692926
3483064
===============reading document enfull5================
<Element '{http://www.mediawiki.org/xml/export-0.10/}mediawiki' at 0x00000127FEF768B0>
title_page change
341641
571372
id change
3483064
4225023
===============reading document enfull6================
<Element '{http://www.mediawiki.org/xml/export-0.10/}mediawiki' at 0x000001289491C180>
title_page change
571372
835237
id ch

# =============生成稀疏距阵和马尔可夫距阵=====================

In [5]:
import scipy.sparse as ss

st = time.time()
tmat = ss.lil_matrix((id,id))


for i in G.edges():
    iid = G.nodes[i[0]]["id"]
    jid = G.nodes[i[1]]["id"]
    tmat[iid,jid] = 1
et = time.time()

print("for making using time %f" %(et - st))
print(id)

for making using time 50.969956
7959698


In [6]:
mat = tmat
count = 0

#print(mat.data)
for i in mat.data:
    if i != []:
        for j in range(len(i)):
            mat.data[count][j] = mat.data[count][j] / float(len(i))
    count = count + 1
            
mat = mat.tocsr()
#print(mat)

# =================计算迭代过程================

In [7]:
import numpy as np

M = copy.deepcopy(mat)

v = np.full((id,1),float(1/id))

beta = 0.85

e = 1e-200

#print(v1.toarray())

i = 1
while True:
    print("epoch: %d" %i)

    st = time.time()
    v1 = M.T.dot(v)*beta + (1-beta) * 1/id
    
    d = v1 - v
    
    print("counting different value")
    
    d = np.linalg.norm(d)
    print("different {0}".format(d))
    if d < e:
        break
            
    et = time.time()

    print("using time %f" %(et-st))
    
    print("not convergence")
    
    v = v1
    i = i + 1
    print("==============================================")
    
print("finish")
#v = v.reshape(1,-1)


epoch: 1
counting different value
different 0.00027872747375708694
using time 0.177637
not convergence
epoch: 2
counting different value
different 5.473169273761652e-05
using time 0.108024
not convergence
epoch: 3
counting different value
different 6.753427822674328e-06
using time 0.107024
not convergence
epoch: 4
counting different value
different 8.244714693892644e-07
using time 0.097021
not convergence
epoch: 5
counting different value
different 1.8904841216152696e-07
using time 0.108576
not convergence
epoch: 6
counting different value
different 7.878347167213418e-08
using time 0.099022
not convergence
epoch: 7
counting different value
different 4.700894507881941e-08
using time 0.103023
not convergence
epoch: 8
counting different value
different 3.529574315656022e-08
using time 0.102537
not convergence
epoch: 9
counting different value
different 5.229572678737019e-09
using time 0.105591
not convergence
epoch: 10
counting different value
different 4.082621301233591e-09
using time 0.

# ====================输出过程=======================

In [8]:
out = []

for i in v1:
    out.append([float(i)])


count = 0
for i in G.nodes():
    out[count].append(i)
    count = count + 1  

out.sort(key = lambda x:x[0],reverse = True)    

f = open("result_pagerank_ten.txt","w",encoding="utf-8")
f.write("title pages {1}\ntotal pages {0}\n".format(id,title_page))
f.write("=====================================================\n")
for i in out:
    f.write(str(i[0])+"  "+str(i[1])+"\n")

f.close()

print("title pages {1}\ntotal pages {0}\n".format(id,title_page))
print("finish")

title pages 2367488
total pages 7959698

finish


# 標準pagerank計算

In [9]:
pagerank_list = nx.pagerank_scipy(G,alpha = 0.85,tol = 1e-200)
pagerank_list = sorted(pagerank_list.items(),key = lambda x:x[1],reverse = True)

with open("package_ten.txt",'w',encoding="utf8") as o:
    for i in pagerank_list:
        o.write(i[0]+"   {0}\n".format(i[1]))
    